In [9]:
import numpy as np
from glob import glob
from bcg import load_bcg
from fs import load_ecg
import plotly.graph_objs as go
from plotly.offline import plot
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
data_dir = "/home/guo/physio/BCG_data/bcg_ecg_data/bcg_ecg_data(500HZ)/"

In [11]:
bcg_files = glob(data_dir+"*.txt", recursive=False)
ecg_files = glob(data_dir+"*.edf", recursive=False)

In [12]:
bcg_files.sort()
ecg_files.sort()

In [15]:
from bcg import load_bcg
import findpeaks3
import autopeaks
import pdb
from findpeaks3 import transform_signal_by_energy

def findpeaks_in_bcg_resp(signal):
    fs = 500
    min_dist = int(2*fs)
    buffer_size = 15*fs
    bcg_auto_peaks = autopeaks.AutoPeaks(thres=0.65,min_dist=min_dist,buffer_size=buffer_size)
    list(map(bcg_auto_peaks.findpeaks, signal))
    return bcg_auto_peaks.peak_indexes, bcg_auto_peaks.peak_values

def findpeaks_in_ecg_resp(signal):
    fs = 500
    min_dist = int(0.6*3*fs)
    buffer_size = 10*fs
    ecg_auto_peaks = autopeaks.AutoPeaks(thres=0.7,min_dist=min_dist,buffer_size=buffer_size)
    list(map(ecg_auto_peaks.findpeaks, signal))
    return ecg_auto_peaks.peak_indexes, ecg_auto_peaks.peak_values

def findpeaks_for_bcg_and_ecg(bcg_file, ecg_file, offset=0, is_return=False):
    if offset >= 0:
        bcg_offset = offset
    else:
        bcg_offset = 0
    bcg = load_bcg(bcg_file, offset=bcg_offset,resp=True)
    bcg = np.asarray(bcg)
    window_size = 1.5
    bcg = transform_signal_by_energy(bcg,window_size=window_size,fs=500)
    bcg = bcg / 10000000
    bcg -= np.median(bcg)
    ecg = load_ecg(ecg_file, resp=True)
    ecg = np.asarray(ecg)
    ecg = ecg / 100
    ecg -= np.median(ecg)
    if offset < 0:
        ecg = ecg[-offset:]
    #print(bcg[:100])
    start = 100000
    end = start + 400000
    #bcg = bcg[start:end]
    #ecg = ecg[start:end]
    bcg_peak_indices, bcg_peak_values = findpeaks_in_bcg_resp(np.asarray(bcg))
    ecg_peak_indices, ecg_peak_values = findpeaks_in_ecg_resp(ecg)
    if True:
        plot(go.Figure(data=[go.Scatter(x=bcg_peak_indices[1:],
                                        y=30000/np.diff(bcg_peak_indices),  
                                        mode="lines+markers",
                                        name="bcg_resp_intervals",
                                        marker=dict(color="blue")),
                             go.Scatter(x=ecg_peak_indices[1:], 
                                        y=30000/np.diff(ecg_peak_indices), 
                                        mode="lines+markers", 
                                        name="ecg_intervals",
                                        marker=dict(color="green")),
                             go.Scatter(x=ecg_peak_indices[1:], 
                                        y=30000/np.diff(ecg_peak_indices) + 3, 
                                        mode="lines+markers", 
                                        name="ecg_intervals_upper",
                                        marker=dict(color="black")),
                             go.Scatter(x=ecg_peak_indices[1:], 
                                        y=30000/np.diff(ecg_peak_indices) - 3, 
                                        mode="lines+markers", 
                                        name="ecg_intervals_lower",
                                        marker=dict(color="black")),],
                       layout=go.Layout(yaxis=dict(range=[3,30]))),filename=bcg_file+str(window_size))
    if True:
        plot([go.Scatter(y=bcg, name="bcg"),
              go.Scatter(x=bcg_peak_indices, y=bcg_peak_values, mode="markers", name="bcg resp interval"),
              go.Scatter(y=ecg, name="ecg"),
              go.Scatter(x=ecg_peak_indices, y=ecg_peak_values, mode="markers",name="ecg resp interval")],filename="resp rate")

In [16]:
import time


def get_bcg_resp_offset(bcg_file):
    offset = 0
    if "yjj" in bcg_file:
        offset = 39558-396
    elif "glt" in bcg_file:
        offset = 36621 + 39421 - 116 - 336
    elif "hwm" in bcg_file:
        offset = 1000 + 74953 - 169
    elif "lcc" in bcg_file:
        offset = 60000 + 8287 - 10
    elif "lifeng" in bcg_file:
        offset = 90000 - 25471 + 4396
    elif "zyh" in bcg_file:
        offset = 300000 - 14583
    elif "zcj" in bcg_file:
        offset = -10235 + 170
    elif "wkp" in bcg_file:
        offset = 5004
    return offset


for bcg_file,ecg_file in zip(bcg_files, ecg_files):
    if "lifeng" not in bcg_file:
        continue
    print(bcg_file)
    offset = get_bcg_resp_offset(bcg_file)
    print("offset: {}".format(offset))
    findpeaks_for_bcg_and_ecg(bcg_file, ecg_file, offset=offset)

/home/guo/physio/BCG_data/bcg_ecg_data/bcg_ecg_data(500HZ)/glt_bcg_wave(500HZ)(03-15 142141).txt
offset: 75590


/home/guo/anaconda3/lib/python3.6/site-packages/scipy/signal/signaltools.py:2383: FutureWarning:

Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.

/home/guo/anaconda3/lib/python3.6/site-packages/plotly/offline/offline.py:451: UserWarning:

Your filename `/home/guo/physio/BCG_data/bcg_ecg_data/bcg_ecg_data(500HZ)/glt_bcg_wave(500HZ)(03-15 142141).txt1.5` didn't end with .html. Adding .html to the end of your file.

/home/guo/anaconda3/lib/python3.6/site-packages/plotly/offline/offline.py:451: UserWarning:

Your filename `resp rate` didn't end with .html. Adding .html to the end of your file.



/home/guo/physio/BCG_data/bcg_ecg_data/bcg_ecg_data(500HZ)/hwm_bcg_wave(500HZ)(03-15 200629).txt
offset: 75784


/home/guo/anaconda3/lib/python3.6/site-packages/scipy/signal/signaltools.py:2383: FutureWarning:

Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.

/home/guo/anaconda3/lib/python3.6/site-packages/plotly/offline/offline.py:451: UserWarning:

Your filename `/home/guo/physio/BCG_data/bcg_ecg_data/bcg_ecg_data(500HZ)/hwm_bcg_wave(500HZ)(03-15 200629).txt1.5` didn't end with .html. Adding .html to the end of your file.

/home/guo/anaconda3/lib/python3.6/site-packages/plotly/offline/offline.py:451: UserWarning:

Your filename `resp rate` didn't end with .html. Adding .html to the end of your file.



/home/guo/physio/BCG_data/bcg_ecg_data/bcg_ecg_data(500HZ)/lcc_bcg_wave(500HZ)(03-15 141708).txt
offset: 68277


...

/home/guo/anaconda3/lib/python3.6/site-packages/scipy/signal/signaltools.py:2383: FutureWarning:

Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.

/home/guo/anaconda3/lib/python3.6/site-packages/plotly/offline/offline.py:451: UserWarning:

Your filename `/home/guo/physio/BCG_data/bcg_ecg_data/bcg_ecg_data(500HZ)/yjj_bcg_wave(500HZ)(03-15 145935).txt1.5` didn't end with .html. Adding .html to the end of your file.

/home/guo/anaconda3/lib/python3.6/site-packages/plotly/offline/offline.py:451: UserWarning:

Your filename `resp rate` didn't end with .html. Adding .html to the end of your file.



/home/guo/physio/BCG_data/bcg_ecg_data/bcg_ecg_data(500HZ)/zcj_bcg_wave(500HZ)(03-15 154123).txt
offset: -10065


/home/guo/anaconda3/lib/python3.6/site-packages/scipy/signal/signaltools.py:2383: FutureWarning:

Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.

/home/guo/anaconda3/lib/python3.6/site-packages/plotly/offline/offline.py:451: UserWarning:

Your filename `/home/guo/physio/BCG_data/bcg_ecg_data/bcg_ecg_data(500HZ)/zcj_bcg_wave(500HZ)(03-15 154123).txt1.5` didn't end with .html. Adding .html to the end of your file.

/home/guo/anaconda3/lib/python3.6/site-packages/plotly/offline/offline.py:451: UserWarning:

Your filename `resp rate` didn't end with .html. Adding .html to the end of your file.



/home/guo/physio/BCG_data/bcg_ecg_data/bcg_ecg_data(500HZ)/zyh_bcg_wave(500HZ)(05-10 132226).txt
offset: 285417


/home/guo/anaconda3/lib/python3.6/site-packages/scipy/signal/signaltools.py:2383: FutureWarning:

Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.

/home/guo/anaconda3/lib/python3.6/site-packages/plotly/offline/offline.py:451: UserWarning:

Your filename `/home/guo/physio/BCG_data/bcg_ecg_data/bcg_ecg_data(500HZ)/zyh_bcg_wave(500HZ)(05-10 132226).txt1.5` didn't end with .html. Adding .html to the end of your file.

/home/guo/anaconda3/lib/python3.6/site-packages/plotly/offline/offline.py:451: UserWarning:

Your filename `resp rate` didn't end with .html. Adding .html to the end of your file.



In [47]:
for bcg_file,ecg_file in zip(bcg_files, ecg_files):
    if "glt" not in bcg_file:
        continue
    print(bcg_file)
    offset = get_bcg_resp_offset(bcg_file)
    signal = load_bcg(bcg_file, offset=offset)
    start = 100000
    end = start + 100000
    signal = signal[start:end]  
    signal_low = lowpass_filter(signal, fs=500)
    signal_med15 = medfilt(signal, kernel_size=51)
    signal_med15_low = lowpass_filter(signal_med15, fs=500)
    
    ecg = load_ecg(ecg_file, resp=True)
    ecg = ecg[start:end]
    ecg = ecg - np.median(ecg)
    ecg = np.asarray(ecg) / 10
    
    def get_signal_energy(signal_low, std=1000000):
        signal_low_energy = transform_signal_by_energy(signal_low, window_size=1.5,fs=500)
        signal_low_energy -= np.median(signal_low_energy)
        signal_low_energy /= 500
        return signal_low_energy
    
    signal_low_energy = get_signal_energy(signal_low)
    signal_med15_low_energy = get_signal_energy(signal_med15_low)
    print(len(signal_low_energy))
    plot([go.Scatter(y=signal, name="signal"),
          go.Scatter(y=signal_low, name="signal_low"),
          go.Scatter(y=signal_low_energy, name="signal_low_engery"),
          go.Scatter(y=signal_med15, name="signal_med15"),
          go.Scatter(y=signal_med15_low, name="signal_med15_low"),
          go.Scatter(y=signal_med15_low_energy, name="signal_med15_low_engery"),
          go.Scatter(y=ecg, name="ecg"),])

/home/guo/physio/BCG_data/bcg_ecg_data/bcg_ecg_data(500HZ)/glt_bcg_wave(500HZ)(03-15 142141).txt


/home/guo/anaconda3/lib/python3.6/site-packages/scipy/signal/signaltools.py:2383: FutureWarning:

Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.



99251


In [28]:
from fs import lowpass_filter

In [34]:
from findpeaks3 import transform_signal_by_energy